In [32]:
import cloudknot as ck
ck.set_region('us-west-2')

In [35]:
def afq_cam_can(subject):
    import fury
    import xvfbwrapper
    import AFQ.data as afqd
    import AFQ.api as api
    import AFQ.mask as afm
    import os.path as op
    import s3fs

    local_bids_folder = "./cam_can"
    bucket = "cam-can-mri"
    s3_prefix = ""
    s3_prefix_qsiprep = "derivatives/qsiprep"
    scope = 'qsiprep'

    # select subjects from qsiprep
    study_qsiprep = afqd.S3BIDSStudy(
        "Cam-CAN-qsiprep",
        bucket,
        s3_prefix_qsiprep,
        subjects=[subject],
        anon=False,
        random_seed=42)

    study = afqd.S3BIDSStudy(
        "Cam-CAN",
        bucket,
        s3_prefix,
        subjects=study_qsiprep.subjects[0].subject_id,
        anon=False,
        random_seed=42)
    study.download(
        local_bids_folder,
        include_derivs=scope)
    print(study_qsiprep.subjects[0]._files['derivatives'])
    
    tracking_params = {
        "n_seeds": 1, # also try 2
        "directions": "prob",
        "odf_model": "CSD", # also try dki
        "seed_mask": afm.MaskFile("WM_probseg", scope=scope), 
        "seed_treshold": 0.1,
        "stop_mask": afm.ScalarMask("dki_fa"),
        "stop_threshold": 0.2,
    }
    myafq = api.AFQ(
        local_bids_folder,
        dmriprep=scope,
        brain_mask=afm.MaskFile("brain_mask", scope=scope),
        scalars=['dki_md', 'dki_fa'],
        tracking_params=tracking_params,
        virtual_frame_buffer=True)
    myafq.export_all()

    s3fs.S3FileSystem().put(
        op.join(local_bids_folder, "afq"),
        op.join(bucket, s3_prefix, "afq"),
        recursive=True)
    

In [36]:
afq_cam_can(1)

/home/john/miniconda3/envs/afq/lib/python3.7/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


Retrieving subject S3 keys
[########################################] | 100% Completed |  0.4s
Retrieving subject S3 keys
[########################################] | 100% Completed |  0.3s



{}


KeyboardInterrupt: 

In [4]:
afq_knot = ck.Knot(name='afq_cam_can-64gb-20819-1',
                  func=afq_cam_can,
                  base_image='libglxvfb:1',
                  image_github_installs="https://github.com/36000/pyAFQ.git@mask_files",
                  pars_policies=('AmazonS3FullAccess',),
                  resource_type="SPOT",
                  bid_percentage=100,
                  memory=64000)

KeyboardInterrupt: 

In [13]:
inputs = [1, 2, 3]

In [14]:
ft = afq_knot.map(inputs)

In [15]:
afq_knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
d11e535b-97b5-47db-85a9-00637c1b556c        afq-cam-can-64gb-20819-1-0        SUBMITTED
